# 🧠 Text Toxicity Moderation Model
**Author:** Neptune Michel  
**Environment:** Jupyter Notebook  
**Objective:** Detect whether a piece of text is toxic or non-toxic using NLP techniques.

## 🧭 1. Introduction
This project demonstrates how to detect toxic comments (e.g., hate speech, insults, profanity, harassment) using NLP.

In [ ]:
# ⚙️ Setup and Imports
!pip install pandas numpy scikit-learn matplotlib seaborn transformers torch gradio SpeechRecognition --quiet